## Getting PBP

In [ ]:
import pandas as pd
import numpy as np
import os
from scipy.stats import percentileofscore as perc
from simFootballPBP import *

'''
Functions
'''

def scoreDiff(row):
    if row['recTeam'] == row['awayTeam']:
        return row['homeScore'] - row['awayScore']
    else:
        return row['awayScore'] - row['homeScore']

def secSinceHalf(row):
    return 1800 - int(row['totTime'])

def fieldPosScore(row):
    if row['dist2goal'] >= 60:
        return 1.0
    elif row['dist2goal'] >= 50:
        return 1.1 ** (40 - row['dist2goal'])
    else:
        return 1.1**10 * 1.2**(50 - row['dist2goal'])

def distScore(row):
    if int(row['distance']) >= 10:
        return 0.2
    elif 7 <= row['distance'] <= 9:
        return 0.4
    elif 4 <= row['distance'] <= 6:
        return 0.6
    elif 2 <= row['distance'] <= 3:
        return 0.8
    else:
        return 1.0

def scoreDiffMult(pointdiff):
    if pointdiff > 0:
        return 1
    elif pointdiff == 0:
        return 2
    elif pointdiff < -8:
        return 3
    else:
        return 4

def timeMult(row):
    sec = secSinceHalf(row)
    PD = scoreDiff(row)
    
    if PD <= 0 and sec >= 0:
        return ((sec * 0.001)**3) + 1.
    else:
        return 1.

def surrenderIndex(row):
    pd = scoreDiff(row)
    return fieldPosScore(row) * distScore(row) * scoreDiffMult(pd) * timeMult(row)

def receiving(row):
    if row['teamPoss'] == row['homeTeam']:
        return row['awayTeam']
    else:
        return row['homeTeam']
    
def touchback(row):
    if 'Touchback' in row['play']:
        return "Touchback"
    elif 'No return' in row['play']:
        return "No return"
    else:
        return "Returned"

def puntDist(row):
    if 'BLOCKED' in row.play:
        return 0
    elif 'Touchback' in row.play:
        return row.dist2goal - 20
    else:
        return int(row.play.split('of ')[-1].split(' yards')[0])
    
def stringify(row):
    num = ['st','nd','rd','th']
    if row['awayTeam'] == row['teamPoss']:
        puntScore = row['awayScore']
        recScore = row['homeScore']
    else:
        puntScore = row['homeScore']
        recScore = row['awayScore']
    return "Q%s - %s - %i%s and %i\n%s %s - %s %s\n"%(row['Q'],row['time'],row['down'],num[int(row['down'])-1],row['distance'],row['teamPoss'],puntScore,row['recTeam'],recScore)


In [ ]:
S = 25

sData = []
idList, idDict = getSeasonIDs(S)

print('Season:',S)
print('Games:',len(idList))

In [3]:
print('Gathering PBP...')
for i in idList:
    sData.append(getGameData(S,i,idDict))

sDF = pd.concat(sData)
sDF.to_csv('PBP/S%sPBP.csv'%S)

print('PBP for S%s saved!'%S)

Gathering PBP...
PBP for S25 saved!


In [5]:
print('Combining all Seasons...')
import os
allDF = pd.concat([pd.read_csv('PBP/%s'%p) for p in next(os.walk('PBP'))[2] if '.csv' in p])
allDF = allDF.reset_index().sort_values('S')
# allDF.to_csv('allPBP.csv')

print('Getting Punts...')
puntDF = allDF[allDF['play'].str.contains('Punt ')]

print('Done!')

Combining all Seasons...
Getting Punts...
Done!


In [6]:
print('Cleaning up PuntDF...')
NFL = np.load("2009-2018_surrender_indices.npy")
puntDF['recTeam'] = puntDF.apply(lambda row : receiving(row),axis=1)
puntDF['surrenderIndex'] = puntDF.apply(lambda row : surrenderIndex(row),axis=1)
puntDF = puntDF[['S','W','gameID','Q','time','awayTeam','awayScore','homeScore','homeTeam'
                 ,'teamPoss','recTeam','down','distance','side','dist2goal','surrenderIndex','play']]
puntDF = puntDF.sort_values('surrenderIndex',ascending=False)
puntDF['NFLpercentiles'] = puntDF.apply(lambda row : perc(NFL,row['surrenderIndex']),axis=1)
puntDF['percentiles'] = puntDF.apply(lambda row : perc(puntDF['surrenderIndex'],row['surrenderIndex']),axis=1)
puntDF['result'] = puntDF.apply(lambda row : touchback(row),axis = 1)
puntDF['scoreDiff'] = puntDF.apply(lambda row : scoreDiff(row),axis = 1)
puntDF['puntDist'] = puntDF.apply(lambda row : puntDist(row),axis = 1)
puntDF['puntEndLoc'] = puntDF['dist2goal'] - puntDF['puntDist']
puntDF['surrenderRank'] = puntDF.surrenderIndex.rank(method='max',ascending=False).astype('int')
puntDF['situation'] = puntDF.apply(lambda row : stringify(row), axis = 1)

puntDF = puntDF[['S', 'W', 'gameID', 'Q', 'time', 'awayTeam', 'awayScore', 'homeScore',
       'homeTeam', 'teamPoss', 'recTeam', 'down', 'distance', 'dist2goal', 'puntDist',
       'puntEndLoc', 'surrenderIndex','surrenderRank', 'percentiles', 'NFLpercentiles', 'play',
       'situation','result']]

print('Exporting Surrender DF...')
puntDF.to_csv('surrender.csv')

print('Done!')

Cleaning up PuntDF...


/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/mike/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Exporting Surrender DF...
Done!


## Bot Functions

In [14]:
s25 = pd.read_csv('PBP/S25PBP.csv')

In [30]:
s25['play'].str.replace('Dasistwirklichseinnachname, A.','D, Alex')

0                                      D, Alex kicks off.
1                        The kick sails into the endzone.
2                PickSix, H. takes it out of the endzone!
3       Kickoff of 77 yards. Returned by PickSix, H. f...
4         Jack, M. SACKED by Rapid Eagle - DT for -7 yds.
                              ...                        
2928    Rush by Z, D. for 2 yds. Tackle by Annastesia,...
2929    Rush by Skyline, A. for 4 yds. Tackle by De Vi...
2930    Rush by Z, D. for a short gain. Tackle by Harr...
2931        Punt by Kokot (R), B. of 39 yards. Touchback.
2932    Pass by Fujiwara (C), C., complete to Howlett ...
Name: play, Length: 2933, dtype: object

In [17]:
test = s25[(s25.S == 25) & (s25.W == 1) & ((s25.homeTeam == 'COL') | (s25.awayTeam == 'COL'))]

In [26]:
string = test.play.iloc[0].split('yards. ')
print(str(string[0] + 'yards.\n' + string[1]))

Punt by Small, J. of 67 yards.
No return.


In [29]:
import tabulate
tabulate(test[['surrenderRank','situation', 'play','surrenderIndex','percentile']],
         headers=["Rank","Game Situation","Punt","Index","Perc."],tablefmt='psql',showindex=False)

KeyError: "['situation', 'percentile', 'surrenderIndex', 'surrenderRank'] not in index"

## S3

In [ ]:
import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

In [ ]:
s3.download_file('isfl-surrender-bot','surrender.csv','testsurrender.csv')

In [95]:
curSW = pd.read_csv('curSW.csv').iloc[0]

In [96]:
testDF = pd.read_csv('testsurrender.csv')

In [97]:
if (curSW.S == curS) and (curSW.W == curW):
    print('No update.')
else:
    print('update')

No update.


In [ ]:
curS = max(testDF.S)
curW = max(testDF[testDF.S == curS].W)
print(curS,curW)

In [ ]:
pd.DataFrame([curS,curW])

In [83]:
thisWeek = testDF[(testDF.S == curS) & (testDF.W == curW) & (testDF.percentiles >= 90)].sort_values("percentiles")
thisWeek.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5 entries, 2083 to 169
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      5 non-null      int64  
 1   S               5 non-null      int64  
 2   W               5 non-null      int64  
 3   gameID          5 non-null      int64  
 4   Q               5 non-null      int64  
 5   time            5 non-null      object 
 6   awayTeam        5 non-null      object 
 7   awayScore       5 non-null      int64  
 8   homeScore       5 non-null      int64  
 9   homeTeam        5 non-null      object 
 10  teamPoss        5 non-null      object 
 11  recTeam         5 non-null      object 
 12  down            5 non-null      float64
 13  distance        5 non-null      float64
 14  dist2goal       5 non-null      float64
 15  puntDist        5 non-null      float64
 16  puntEndLoc      5 non-null      float64
 17  surrenderIndex  5 non-null      fl

In [84]:
thisWeek

,Unnamed: 0,S,W,gameID,Q,time,awayTeam,awayScore,homeScore,homeTeam,...,dist2goal,puntDist,puntEndLoc,surrenderIndex,surrenderRank,percentiles,NFLpercentiles,play,situation,result
2083,281985,25,7,6414,4,8:40,NY,16,24,NO,...,61.0,41.0,20.0,12.388608,2084,90.018210,94.515940,"Punt by Smalls, D. of 61 yards.\n Touchback.",Q4 - 8:40 - 4th and 1\nNY 16 - NO 24\n,Touchback
1199,282050,25,7,6415,1,0:17,BAL,14,0,SAR,...,35.0,15.0,20.0,23.977108,1200,94.263945,97.824656,"Punt by Small, J. of 35 yards.\n Touchback.",Q1 - 0:17 - 4th and 12\nSAR 0 - BAL 14\n,Touchback
357,281355,25,7,6411,4,6:30,SJS,24,16,COL,...,45.0,25.0,20.0,78.547908,358,98.289247,99.795262,"Punt by Banana, S. of 45 yards.\n Touchback.",Q4 - 6:30 - 4th and 2\nCOL 16 - SJS 24\n,Touchback
331,281996,25,7,6414,4,4:45,NY,16,24,NO,...,38.0,18.0,20.0,82.833245,332,98.413839,99.806230,"Punt by Smalls, D. of 38 yards.\n Touchback.",Q4 - 4:45 - 4th and 14\nNY 16 - NO 24\n,Touchback
169,282200,25,7,6416,1,12:18,CHI,0,0,PHI,...,31.0,11.0,20.0,132.583814,177,99.173376,99.912255,"Punt by Fencik (R), J. of 31 yards.\n Touchback.",Q1 - 12:18 - 4th and 3\nPHI 0 - CHI 0\n,Touchback


In [85]:
def tweetMessage(row):
    
    # location of ball
    if row.dist2goal < 50:
        loc = "%s %i"%(row.recTeam,row.dist2goal)
    elif row.dist2goal == 50:
        loc = "midfield"
    else:
        loc = "%s %i"%(row.teamPoss,100-row.dist2goal)
    
    # who has what score
    if row['awayTeam'] == row['teamPoss']:
        puntScore = row['awayScore']
        recScore = row['homeScore']
    else:
        puntScore = row['homeScore']
        recScore = row['awayScore']
    
    # score string
    if puntScore < recScore:
        rel = "losing"
    elif puntScore > recScore:
        rel = "winning"
    else:
        rel = "tied"
        
    # percentile string
    percentile = int(row.percentiles)
    mod = percentile%10
    if mod == 1:
        perc = "%ist"%percentile
    elif mod == 2:
        perc = "%ind"%percentile
    elif mod == 3:
        perc = "%ird"%percentile
    else:
        perc = "%ith"%percentile
    
    # down and distance
    if row.down == 1:
        dW = "%ist"%row.down
    elif row.down == 2:
        dW = "%ind"%row.down
    elif row.down == 3:
        dW = "%ird"%row.down
    else:
        dW = "%ith"%row.down
    
    # quarter
    if row.Q == 1:
        Q = "%ist"%row.Q
    elif row.Q == 2:
        Q = "%ind"%row.Q
    elif row.Q == 3:
        Q = "%ird"%row.Q
    else:
        Q = "%ith"%row.Q
    
    string = "In S%i W%i, %s decided to punt to %s from the %s on %s and %i with %s remaining in the %s quarter while %s %i to %i.\n\nWith a Surrender Index of %.2f, this punt ranks at the %s percentile of cowardly punts in ISFL History. Overall, it is ranked #%i all time."%(row.S,row.W,row.teamPoss,row.recTeam,loc,dW,row.distance,row.time,Q,rel,puntScore,recScore,row.surrenderIndex,perc,row.surrenderRank)
    return string


In [86]:
thisWeek['tweet'] = thisWeek.apply(lambda x : tweetMessage(x),axis=1)

In [87]:
for i in range(len(thisWeek)):
    print(thisWeek.tweet.iloc[i])
    print('\n(%i char)'%len(thisWeek.tweet.iloc[i]))
    print('--')

In S25 W7, NY decided to punt to NO from the NY 39 on 4th and 1 with 8:40 remaining in the 4th quarter while losing 16 to 24.

With a Surrender Index of 12.39, this punt ranks at the 90th percentile of cowardly punts in ISFL History. Overall, it is ranked #2084 all time.

(271 char)
--
In S25 W7, SAR decided to punt to BAL from the BAL 35 on 4th and 12 with 0:17 remaining in the 1st quarter while losing 0 to 14.

With a Surrender Index of 23.98, this punt ranks at the 94th percentile of cowardly punts in ISFL History. Overall, it is ranked #1200 all time.

(274 char)
--
In S25 W7, COL decided to punt to SJS from the SJS 45 on 4th and 2 with 6:30 remaining in the 4th quarter while losing 16 to 24.

With a Surrender Index of 78.55, this punt ranks at the 98th percentile of cowardly punts in ISFL History. Overall, it is ranked #358 all time.

(273 char)
--
In S25 W7, NY decided to punt to NO from the NO 38 on 4th and 14 with 4:45 remaining in the 4th quarter while losing 16 to 24.

With a